# 피처 엔지니어링
## 1.데이터 합치기

In [1]:
import pandas as pd

data_path = '/kaggle/input/porto-seguro-safe-driver-prediction/'
train = pd.read_csv(data_path + 'train.csv', index_col="id")
test = pd.read_csv(data_path + 'test.csv', index_col="id")
submission = pd.read_csv(data_path + 'sample_submission.csv', index_col="id")

In [2]:
all_data = pd.concat([train, test], ignore_index=True)
all_data = all_data.drop("target", axis=1) # 타깃값 제거

In [3]:
all_features = all_data.columns # 전체 피처
all_features

Index(['ps_ind_01', 'ps_ind_02_cat', 'ps_ind_03', 'ps_ind_04_cat',
       'ps_ind_05_cat', 'ps_ind_06_bin', 'ps_ind_07_bin', 'ps_ind_08_bin',
       'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_12_bin',
       'ps_ind_13_bin', 'ps_ind_14', 'ps_ind_15', 'ps_ind_16_bin',
       'ps_ind_17_bin', 'ps_ind_18_bin', 'ps_reg_01', 'ps_reg_02', 'ps_reg_03',
       'ps_car_01_cat', 'ps_car_02_cat', 'ps_car_03_cat', 'ps_car_04_cat',
       'ps_car_05_cat', 'ps_car_06_cat', 'ps_car_07_cat', 'ps_car_08_cat',
       'ps_car_09_cat', 'ps_car_10_cat', 'ps_car_11_cat', 'ps_car_11',
       'ps_car_12', 'ps_car_13', 'ps_car_14', 'ps_car_15', 'ps_calc_01',
       'ps_calc_02', 'ps_calc_03', 'ps_calc_04', 'ps_calc_05', 'ps_calc_06',
       'ps_calc_07', 'ps_calc_08', 'ps_calc_09', 'ps_calc_10', 'ps_calc_11',
       'ps_calc_12', 'ps_calc_13', 'ps_calc_14', 'ps_calc_15_bin',
       'ps_calc_16_bin', 'ps_calc_17_bin', 'ps_calc_18_bin', 'ps_calc_19_bin',
       'ps_calc_20_bin'],
      dtype='obj

## 2.명목형 피처 원-핫 인코딩

In [4]:
from sklearn.preprocessing import OneHotEncoder

cat_features = [feature for feature in all_features if "cat" in feature] # 명목형 피처 추출
onehot_encoder = OneHotEncoder() # 원-핫 인코더 객체 생성
encoded_cat_matrix = onehot_encoder.fit_transform(all_data[cat_features])
encoded_cat_matrix

<1488028x184 sparse matrix of type '<class 'numpy.float64'>'
	with 20832392 stored elements in Compressed Sparse Row format>

## 3.파생 피처 추가

In [5]:
# "데이터 하나당 결측값 개수"를 파생 피처로 추가
all_data["num_missing"] = (all_data==-1).sum(axis=1)

In [6]:
# 명목형 피처, calc 분류의 피처를 제외한 피처
remaining_features = [feature for feature in all_features 
                      if ("cat" not in feature and "calc" not in feature)]
# num_missing을 remaining_features에 추가
remaining_features.append("num_missing")

In [7]:
# 분류가 ind인 피처
ind_features = [feature for feature in all_features if "ind" in feature]
is_first_feature = True

for ind_feature in ind_features:
    if is_first_feature:
        all_data["mix_ind"] = all_data[ind_feature].astype(str) + "_"
        is_first_feature = False
    else:
        all_data["mix_ind"] += all_data[ind_feature].astype(str) + "_"

In [8]:
# 명목형 피처의 고윳값별 개수
cat_count_features = []

for feature in cat_features+["mix_ind"]:
    val_counts_dict = all_data[feature].value_counts().to_dict()
    all_data[f'{feature}_count'] = all_data[feature].apply(lambda x:val_counts_dict[x])
    cat_count_features.append(f'{feature}_count')
cat_count_features

['ps_ind_02_cat_count',
 'ps_ind_04_cat_count',
 'ps_ind_05_cat_count',
 'ps_car_01_cat_count',
 'ps_car_02_cat_count',
 'ps_car_03_cat_count',
 'ps_car_04_cat_count',
 'ps_car_05_cat_count',
 'ps_car_06_cat_count',
 'ps_car_07_cat_count',
 'ps_car_08_cat_count',
 'ps_car_09_cat_count',
 'ps_car_10_cat_count',
 'ps_car_11_cat_count',
 'mix_ind_count']

## 4.필요 없는 피처 제거

In [9]:
from scipy import sparse

# 필요 없는 피처들 제거
drop_features = ["ps_ind_14", "ps_ind_10_bin", "ps_ind_11_bin", "ps_ind_12_bin", "ps_ind_13_bin", "ps_car_14"]
# remaining_features, cat_count_features에서 drop_features를 제거한 데이터
all_data_remaining = all_data[remaining_features+cat_count_features].drop(drop_features, axis=1)
# 데이터 합치기
all_data_sprs = sparse.hstack([sparse.csr_matrix(all_data_remaining), encoded_cat_matrix], format="csr")

## 5.데이터 나누기

In [10]:
num_train = len(train)
X = all_data_sprs[:num_train]
X_test = all_data_sprs[num_train:]
y = train["target"].values

# 평가지표 계산 함수 작성
## 1.정규화된 지니계수 계산 함수

In [11]:
import numpy as np

def eval_gini(y_true, y_pred):
    assert y_true.shape == y_pred.shape
    n_samples = y_true.shape[0] # 데이터 개수
    L_mid = np.linspace(1/n_samples, 1, n_samples) # 대각선 값
    
    # 1) 예측값에 대한 지니계수
    pred_order = y_true[y_pred.argsort()] # y_pred 크기순으로, y_true 값 정렬
    L_pred = np.cumsum(pred_order)/np.sum(pred_order) # 로렌츠 곡선
    G_pred = np.sum(L_mid - L_pred) # 예측값에 대한 지니계수
    # 2) 예측이 완벽할 때 지니계수
    true_order = y_true[y_true.argsort()] # y_true 크기순으로, y_true 값 정렬
    L_true = np.cumsum(true_order)/np.sum(true_order) # 로렌츠 곡선
    G_true = np.sum(L_mid - L_true) # 예측이 완벽할 때 지니계수
    # 3) 정규화된 지니계수
    return G_pred/G_true

## 2.XGBoost용 지니계수

In [12]:
def gini(preds, dtrain):
    labels = dtrain.get_label()
    return "gini", eval_gini(labels, preds) # 반환값

# 하이퍼파라미터 최적화
## 1.데이터셋 준비

In [13]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=0)
bayes_dtrain = xgb.DMatrix(X_train, y_train)
bayes_dvalid = xgb.DMatrix(X_valid, y_valid)

## 2.하이퍼파라미터 범위 설정

In [14]:
# 베이지안 최적화를 위한 하이퍼파라미터 범위
param_bounds = {"max_depth": (4, 8),
                "subsample": (0.6, 0.9),
                "colsample_bytree": (0.7, 1.0),
                "min_child_weight": (5, 7),
                "gamma": (8, 11),
                "reg_alpha": (7, 9),
                "reg_lambda": (1.1, 1.5),
                "scale_pos_weight": (1.4, 1.6)}

# 값이 고정된 하이퍼파라미터
fixed_params = {"objective": "binary:logistic",
                "learning_rate": 0.02,
                "random_state": 1991}

## 3.(베이지안 최적화용) 평가지표 계산함수 작성

In [15]:
def eval_function(max_depth, subsample, colsample_bytree, min_child_weight, 
                  reg_alpha, gamma, reg_lambda, scale_pos_weight):
    '''최적화하려는 평가지표(지니계수) 계산 함수'''
    # 베이지안 최적화를 수행할 하이퍼파라미터
    params = {"max_depth": int(round(max_depth)),
              "subsample": subsample,
              "colsample_bytree": colsample_bytree,
              "min_child_weight": min_child_weight,
              "gamma": gamma,
              "reg_alpha": reg_alpha,
              "reg_lambda": reg_lambda,
              "scale_pos_weight": scale_pos_weight}
    params.update(fixed_params) # 고정된 하이퍼파라미터도 추가
    print("하이퍼파라미터:", params)
    
    # XGBoost 모델 훈련
    xgb_model = xgb.train(params=params,
                          dtrain=bayes_dtrain,
                          num_boost_round=2500,
                          evals=[(bayes_dvalid, "bayes_dvalid")],
                          maximize=True,
                          feval=gini,
                          early_stopping_rounds=200,
                          verbose_eval=False)
    best_iter = xgb_model.best_iteration # 최적 반복 횟수
    preds = xgb_model.predict(bayes_dvalid, iteration_range=(0, best_iter)) # 검증 데이터로 예측 수행
    gini_score = eval_gini(y_valid, preds) # 지니계수 계산
    print(f'지니계수: {gini_score}\n')
    
    return gini_score

## 4.최적화 수행

In [16]:
from bayes_opt import BayesianOptimization

# 베이지안 최적화 객체 생성
optimizer = BayesianOptimization(f=eval_function, pbounds=param_bounds, random_state=0)
# 베이지안 최적화 수행
optimizer.maximize(init_points=3, n_iter=6)

|   iter    |  target   | colsam... |   gamma   | max_depth | min_ch... | reg_alpha | reg_la... | scale_... | subsample |
-------------------------------------------------------------------------------------------------------------------------
하이퍼파라미터: {'max_depth': 6, 'subsample': 0.867531900234624, 'colsample_bytree': 0.8646440511781974, 'min_child_weight': 6.0897663659937935, 'gamma': 10.14556809911726, 'reg_alpha': 7.84730959867781, 'reg_lambda': 1.3583576452266626, 'scale_pos_weight': 1.4875174422525386, 'objective': 'binary:logistic', 'learning_rate': 0.02, 'random_state': 1991}


/opt/conda/lib/python3.10/site-packages/xgboost/training.py:39: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


지니계수: 0.28470429444834217

| 1         | 0.2847    | 0.8646    | 10.15     | 6.411     | 6.09      | 7.847     | 1.358     | 1.488     | 0.8675    |
하이퍼파라미터: {'max_depth': 7, 'subsample': 0.6261387899104622, 'colsample_bytree': 0.9890988281503088, 'min_child_weight': 6.0577898395058085, 'gamma': 9.150324556477333, 'reg_alpha': 8.136089122187865, 'reg_lambda': 1.4702386553170643, 'scale_pos_weight': 1.4142072116395774, 'objective': 'binary:logistic', 'learning_rate': 0.02, 'random_state': 1991}
지니계수: 0.28502030930848077

| 2         | 0.285     | 0.9891    | 9.15      | 7.167     | 6.058     | 8.136     | 1.47      | 1.414     | 0.6261    |
하이퍼파라미터: {'max_depth': 7, 'subsample': 0.8341587528859367, 'colsample_bytree': 0.7060655192320977, 'min_child_weight': 6.7400242964936385, 'gamma': 10.497859536643814, 'reg_alpha': 8.957236684465528, 'reg_lambda': 1.4196634256866894, 'scale_pos_weight': 1.4922958724505864, 'objective': 'binary:logistic', 'learning_rate': 0.02, 'random_state': 1991}
지

/opt/conda/lib/python3.10/site-packages/xgboost/training.py:39: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


지니계수: 0.28409091782139356

| 4         | 0.2841    | 0.8843    | 10.45     | 6.838     | 6.494     | 8.552     | 1.381     | 1.423     | 0.7002    |
하이퍼파라미터: {'max_depth': 7, 'subsample': 0.8535233675350644, 'colsample_bytree': 0.92975858050776, 'min_child_weight': 6.249564429359247, 'gamma': 9.95563546750357, 'reg_alpha': 8.411512219837842, 'reg_lambda': 1.424460008293778, 'scale_pos_weight': 1.5416807226581535, 'objective': 'binary:logistic', 'learning_rate': 0.02, 'random_state': 1991}


...

지니계수: 0.2864072333145655

| 7         | 0.2864    | 0.8818    | 9.013     | 6.927     | 6.992     | 7.641     | 1.356     | 1.449     | 0.6931    |
하이퍼파라미터: {'max_depth': 5, 'subsample': 0.6261564417044092, 'colsample_bytree': 0.8763145220620449, 'min_child_weight': 5.135323353557588, 'gamma': 8.39495450163982, 'reg_alpha': 8.950443047087845, 'reg_lambda': 1.4235649099168255, 'scale_pos_weight': 1.5217625173811569, 'objective': 'binary:logistic', 'learning_rate': 0.02, 'random_state': 1991}


/opt/conda/lib/python3.10/site-packages/xgboost/training.py:39: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


지니계수: 0.2840512946560939

| 8         | 0.2841    | 0.8763    | 8.395     | 4.561     | 5.135     | 8.95      | 1.424     | 1.522     | 0.6262    |
하이퍼파라미터: {'max_depth': 6, 'subsample': 0.857971740304964, 'colsample_bytree': 0.9583821245229369, 'min_child_weight': 6.158305055403563, 'gamma': 9.305332775334449, 'reg_alpha': 8.200928434091152, 'reg_lambda': 1.2571039588093065, 'scale_pos_weight': 1.4700266933495618, 'objective': 'binary:logistic', 'learning_rate': 0.02, 'random_state': 1991}


/opt/conda/lib/python3.10/site-packages/xgboost/training.py:39: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


지니계수: 0.2850853383669918

| 9         | 0.2851    | 0.9584    | 9.305     | 5.594     | 6.158     | 8.201     | 1.257     | 1.47      | 0.858     |


## 5.결과 확인

In [17]:
# 평가함수 점수가 최대일 때 하이퍼파라미터
max_params = optimizer.max["params"]
max_params

{'colsample_bytree': 0.8817801730078565,
 'gamma': 9.013424730095146,
 'max_depth': 6.927417000715145,
 'min_child_weight': 6.992334203641873,
 'reg_alpha': 7.640858389939128,
 'reg_lambda': 1.3562805915715632,
 'scale_pos_weight': 1.449446257931491,
 'subsample': 0.6931141936797243}

In [18]:
# 정수형 하이퍼파라미터 변환
max_params["max_depth"] = int(round(max_params["max_depth"]))
# 값이 고정된 하이퍼파라미터 추가
max_params.update(fixed_params)
max_params

{'colsample_bytree': 0.8817801730078565,
 'gamma': 9.013424730095146,
 'max_depth': 7,
 'min_child_weight': 6.992334203641873,
 'reg_alpha': 7.640858389939128,
 'reg_lambda': 1.3562805915715632,
 'scale_pos_weight': 1.449446257931491,
 'subsample': 0.6931141936797243,
 'objective': 'binary:logistic',
 'learning_rate': 0.02,
 'random_state': 1991}

# 모델 훈련 및 성능 검증
## 1.최적 하이퍼파라미터를 이용해 XGBoost 훈련

In [19]:
from sklearn.model_selection import StratifiedKFold

# 층화 K 폴드 교차 검증기
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1991)
# OOF 방식으로 훈련된 모델로, 검증 데이터 타깃값을 예측한 확률을 담을 1차원 배열
oof_val_preds = np.zeros(X.shape[0])
# OOF 방식으로 훈련된 모델로, 테스트 데이터 타깃값을 예측한 확률을 담을 1차원 배열
oof_test_preds = np.zeros(X_test.shape[0])

In [20]:
# OOF 방식으로 모델 훈련, 검증, 예측
for idx, (train_idx, valid_idx) in enumerate(folds.split(X, y)):
    print('#'*40, f'폴드 {idx+1} / 폴드 {folds.n_splits}', '#'*40) # 각 폴드를 구분하는 문구 출력

    X_train, y_train = X[train_idx], y[train_idx] # 훈련용 데이터
    X_valid, y_valid = X[valid_idx], y[valid_idx] # 검증용 데이터
    dtrain = xgb.DMatrix(X_train, y_train)        # XGBoost 전용 훈련 데이터
    dvalid = xgb.DMatrix(X_valid, y_valid)        # XGBoost 전용 검증 데이터
    dtest  = xgb.DMatrix(X_test)                  # XGBoost 전용 테스트 데이터

    # XGBoost 모델 훈련
    xgb_model = xgb.train(params=max_params, 
                          dtrain=dtrain,
                          num_boost_round=2000,
                          evals=[(dvalid, "valid")],
                          maximize=True,
                          feval=gini,
                          early_stopping_rounds=200,
                          verbose_eval=100)
    
    # 모델 성능이 가장 좋을 때의 부스팅 반복 횟수 저장
    best_iter = xgb_model.best_iteration
    # 테스트 데이터를 활용해 OOF 예측
    oof_test_preds += xgb_model.predict(dtest, iteration_range=(0, best_iter))/folds.n_splits
    # 모델 성능 평가를 위한 검증 데이터 타깃값 예측
    oof_val_preds[valid_idx] += xgb_model.predict(dvalid, iteration_range=(0, best_iter))
    # 검증 데이터 예측 확률에 대한 정규화 지니계수
    gini_score = eval_gini(y_valid, oof_val_preds[valid_idx])
    print(f'폴드 {idx+1} 지니계수: {gini_score}\n')

######################################## 폴드 1 / 폴드 5 ########################################


/opt/conda/lib/python3.10/site-packages/xgboost/training.py:39: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


[0]	valid-logloss:0.67669	valid-gini:0.15993
[100]	valid-logloss:0.19137	valid-gini:0.25223
[200]	valid-logloss:0.15806	valid-gini:0.27838
[300]	valid-logloss:0.15480	valid-gini:0.28841
[400]	valid-logloss:0.15427	valid-gini:0.29312
[500]	valid-logloss:0.15413	valid-gini:0.29557
[600]	valid-logloss:0.15408	valid-gini:0.29677
[700]	valid-logloss:0.15402	valid-gini:0.29821
[800]	valid-logloss:0.15402	valid-gini:0.29871
[900]	valid-logloss:0.15398	valid-gini:0.29901
[1000]	valid-logloss:0.15398	valid-gini:0.29908
[1100]	valid-logloss:0.15397	valid-gini:0.29906
[1200]	valid-logloss:0.15396	valid-gini:0.29903
[1300]	valid-logloss:0.15396	valid-gini:0.29874
[1341]	valid-logloss:0.15397	valid-gini:0.29868
폴드 1 지니계수: 0.299375446272267

######################################## 폴드 2 / 폴드 5 ########################################
[0]	valid-logloss:0.67669	valid-gini:0.12436
[100]	valid-logloss:0.19152	valid-gini:0.23846
[200]	valid-logloss:0.15831	valid-gini:0.26558
[300]	valid-logloss:0.15515	v

## 2.OOF 검증 데이터 지니계수

In [21]:
print("OOF 검증 데이터 지니계수:", eval_gini(y, oof_val_preds))

OOF 검증 데이터 지니계수: 0.2891514695415448


# 예측 및 결과 제출

In [22]:
submission["target"] = oof_test_preds
submission.to_csv('submission.csv')